In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
dataset = pd.read_csv('train.csv')
dataset.drop(dataset.columns[[13, 14, 16,17,18,19,10,14,15]], axis = 1, inplace = True)
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()
#gender
X[:, 0] = le.fit_transform(X[:, 2])
#s11
X[:, 1] = le.fit_transform(X[:, 2])
#s12
X[:, 2] = le.fit_transform(X[:, 2])
#s58
X[:, 5] = le.fit_transform(X[:, 2])

In [47]:
# X=pd.DataFrame(X)
# X[9].replace('l', float(1))
# X[9].replace('o', float(0))

# #extracting numerical values from s54
# X[10].replace('b2', float(0))
# X[10].replace('2b', float(0))
# X[10].replace( 'bb', float(0))
# X[10].replace( 'aa', float(0))
# X[10].replace( 'a2', float(1))
# X[10].replace( '2a', float(1))
# X[10].replace( 'ab', float(3))
# X[10].replace( 'ba', float(3))
# X[10].replace('22', float(2))


# X[11].replace(to_replace = 'kK', value ='0',inplace=True)
# X[11].replace(to_replace = 'kk', value ='0',inplace=True)
# X[11].replace(to_replace = '2K', value ='0',inplace=True)
# X[11].replace(to_replace = 'k2', value ='0',inplace=True)
# X[11].replace(to_replace = 'KK', value ='0',inplace=True)
# X[11].replace(to_replace = 'Kk', value ='0',inplace=True)
# X[11].replace(to_replace = '2k', value ='0',inplace=True)
# X[11].replace(to_replace = 'K2', value ='0',inplace=True)
# X[11].replace(to_replace = '22', value ='1',inplace=True)

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
28317    NaN
28318    NaN
28319    NaN
28320    NaN
28321    NaN
Name: 10, Length: 28322, dtype: object

In [4]:
X[9]

array([1, 1, 1, 'N', 1, 1, 'D', 'B', 0, '1', nan, 'kk', 4.387913174,
       0.837726576, 6, 8.326919845, -32.64105966, 0.012796287,
       -9.326014752, 1.38390784, 2.671842379, 3.940304969, 1.716165394, 0,
       0, 0.212873957, 1], dtype=object)

In [5]:
from sklearn.model_selection import train_test_split
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size = 0.3, random_state = 0)


In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
XTrain = sc.fit_transform(XTrain)
XTest = sc.transform(XTest)

ValueError: could not convert string to float: 'Y'

In [18]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'] )

In [19]:
ann.fit(XTrain, yTrain, batch_size=32, epochs=100)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [56]:
y_pred = ann.predict(XTest)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), yTest.reshape(len(yTest), 1)), 1))

266/266 [==============================] - 0s 483us/step
[[1 1]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [57]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(yTest, y_pred)
print(cm)
accuracy_score(yTest, y_pred)*100

[[6855  233]
 [ 969  440]]


85.85383076379898